In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem

In [3]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [4]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [5]:
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#Drop the 'smiles' column
# df_all = df_all.drop(['smiles'], axis=1)
X_smiles = df_train.smiles.values

X_train = []
counter = 1
for molecule_x in X_smiles:
    m = Chem.MolFromSmiles(molecule_x)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,3,nBits=1024)
    bitArray = np.zeros(1024)
    DataStructs.cDataStructs.ConvertToNumpyArray(fp, bitArray)
    X_train.append(bitArray)
    counter = counter + 1
    if (counter%10000) ==0:
        print(counter)
    

# vals = df_all.values
# X_train = vals[:test_idx]
# X_test = vals[test_idx:]

print(X_train.shape)


# print("Train features:", X_train.shape)
# print("Train gap:", Y_train.shape)
# print("Test features:", X_test.shape)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
X_train = np.array(X_train)
print(X_train.shape)

In [22]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_train)


NameError: name 'Y_pred' is not defined

In [23]:
rmse = np.sqrt(mean_squared_error(Y_train, LR_pred))
print(rmse)

0.158625303514


In [ ]:
X_smiles_test = df_test.smiles.values
X_test = []
counter = 1
for molecule_x in X_smiles_test:
    m = Chem.MolFromSmiles(molecule_x)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,3,nBits=1024)
    bitArray = np.zeros(1024)
    DataStructs.cDataStructs.ConvertToNumpyArray(fp, bitArray)
    X_test.append(bitArray)
    counter = counter + 1
    if (counter%10000) ==0:
        print(counter/10000)
    

# vals = df_all.values
# X_train = vals[:test_idx]
# X_test = vals[test_idx:]

X_test = np.array(X_test)
print(X_test.shape)

In [27]:
Upload_Pred = LR.predict(X_test)

In [28]:
write_to_file("LR_Prediction_New_Features.csv", Upload_Pred)

In [7]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_Upload_pred = RF.predict(X_test)

NameError: name 'X_test' is not defined

In [ ]:
print(RF_Upload_pred.shape)

In [ ]:
LRLasso = linear_model.LassoCV(n_jobs=2)
LRLasso.fit(X_train, Y_train)

LRLassoPred = LRLasso.predict(X_train)

rmse = np.sqrt(mean_squared_error(Y_train, LRLassoPred))
print(rmse)

In [8]:
RF_Train_Predict = RF.predict(X_train)

In [9]:
rmse = np.sqrt(mean_squared_error(Y_train, RF_Train_Predict))
print(rmse)

0.0311686787607


In [10]:
X_smiles_test = df_test.smiles.values
X_test = []
counter = 1
for molecule_x in X_smiles_test:
    m = Chem.MolFromSmiles(molecule_x)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,3,nBits=1024)
    bitArray = np.zeros(1024)
    DataStructs.cDataStructs.ConvertToNumpyArray(fp, bitArray)
    X_test.append(bitArray)
    counter = counter + 1
    if (counter%10000) ==0:
        print(counter/10000)
    

# vals = df_all.values
# X_train = vals[:test_idx]
# X_test = vals[test_idx:]

X_test = np.array(X_test)
print(X_test.shape)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
(824230, 1024)


In [11]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [12]:
RF_Train_Predict = RF.predict(X_test)


In [13]:
write_to_file("RF_Prediction_New_Features.csv", RF_Train_Predict)

In [14]:
print(RF_Train_Predict.shape)

(824230,)


In [15]:
X_smiles = df_train.smiles.values

X_train_2048 = []
counter = 1
for molecule_x in X_smiles:
    m = Chem.MolFromSmiles(molecule_x)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,3,nBits=2048)
    bitArray = np.zeros(2048)
    DataStructs.cDataStructs.ConvertToNumpyArray(fp, bitArray)
    X_train_2048.append(bitArray)
    counter = counter + 1
    if (counter%10000) ==0:
        print(counter)
    

# vals = df_all.values
# X_train = vals[:test_idx]
# X_test = vals[test_idx:]

print(X_train_2048.shape)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
LR_2048 = LinearRegression()
LR_2048.fit(X_train_2048, Y_train)
LR_pred = LR_2048.predict(X_train_2048)